# Import

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os.path
import math

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras import Model

from sklearn import metrics
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

import sys

# Data Load

In [ ]:
!unzip -q classify_angle.zip; rm classify_angle.zip

replace classify_angle/Test/0/221003_150821.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!pwd
!ls

/content
classify_angle	sample_data


In [ ]:
image_dir = Path('./classify_angle/')

In [ ]:
#initialized value  
width=200 #the width of OCT image 
height=200 #the height of OCT image 
batch_size=1 # the number of batch 

In [ ]:
## new
filepaths = pd.Series(list(image_dir.glob(r'Train/*/*.jpg')), name='Filepath').astype(str)
angles = pd.Series(filepaths.apply(lambda x : os.path.split(os.path.split(x)[0])[1]), name='value').astype(float)
train_images = pd.concat([filepaths, angles], axis = 1).sample(frac=1.0, random_state=1).reset_index(drop=True)

filepaths = pd.Series(list(image_dir.glob(r'Test/*/*.jpg')), name='Filepath').astype(str)
angles = pd.Series(filepaths.apply(lambda x : os.path.split(os.path.split(x)[0])[1]), name='value').astype(float)
test_images = pd.concat([filepaths, angles], axis = 1).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [ ]:
print(len(train_images))
train_images.head(5)

1440


,Filepath,value
0,classify_angle/Train/352/221001_225151.jpg,352.0
1,classify_angle/Train/193/221001_222521.jpg,193.0
2,classify_angle/Train/152/221001_233150.jpg,152.0
3,classify_angle/Train/35/221003_162228.jpg,35.0
4,classify_angle/Train/70/221003_162818.jpg,70.0


In [ ]:
print(len(test_images))
test_images.head(5)

360


,Filepath,value
0,classify_angle/Test/252/221003_155022.jpg,252.0
1,classify_angle/Test/181/221003_153832.jpg,181.0
2,classify_angle/Test/296/221003_155742.jpg,296.0
3,classify_angle/Test/237/221003_154752.jpg,237.0
4,classify_angle/Test/215/221003_154412.jpg,215.0


In [ ]:
fig, axes = plt.subplots(nrows=4, ncols=10, figsize=(15, 7),
                        subplot_kw={'xticks': [], 'yticks': []})

for i, ax in enumerate(axes.flat):
    ax.imshow(plt.imread(train_images.Filepath[i]))
    ax.set_title(train_images.value[i], fontsize = 12)
plt.tight_layout(pad=0.5)
plt.show()

NameError: ignored

In [ ]:
train_df=train_images.copy()
test_df=test_images.copy()

In [ ]:
print(train_df.shape)
print(test_df.shape)

(360, 2)
(360, 2)


In [ ]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    # rescale =1./255
#     validation_split=0.2
)
test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    # rescale =1./255
)

In [ ]:
train_data = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='value',
    target_size=(height, width),
    color_mode='rgb',
    class_mode='raw',
    batch_size=batch_size,
    shuffle=True,
    seed=42,
    subset='training',
    # save_to_dir= '..',
    # save_format = 'jpeg'
)
test_data = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='value',
    target_size = (height, width),
    color_mode='rgb',
    class_mode='raw',
    batch_size=batch_size,
    shuffle = False
)
# val_images = train_generator.flow_from_dataframe(
#     dataframe=train_df,
#     x_col='Filepath',
#     y_col='value',
#     target_size = (height, width),
#     color_mode='rgb',
#     class_mode='raw',
#     batch_size=batch_size,
#     shuffle=True,
#     seed=42,
#     subset='validation'
# )

Found 360 validated image filenames.
Found 360 validated image filenames.


# Model

In [ ]:
from tensorflow.keras.applications import ResNet50

In [ ]:
resnet_model = ResNet50(weights=None, input_shape=(height, width, 3), include_top=False)
last = resnet_model.output
x = Flatten()(last)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
output_layer = Dense(1, activation='linear')(x)

resnet_model = Model(resnet_model.input, output_layer)
resnet_model.summary()

In [ ]:
model_path = 'resnet50.h5'

In [ ]:
# resnet_model.reset_states()
input_shape = (200,200,3)

input_tensor = layers.Input(shape = input_shape)

conv1_1 = layers.Conv2D(16, (3,3), padding='same', activation='relu')(input_tensor)
conv1_2 = layers.MaxPool2D()(conv1_1)

conv2_1 = layers.Conv2D(32, (3,3), padding='same', activation='relu')(conv1_2)
conv2_2 = layers.MaxPool2D()(conv2_1)

conv3_1 = layers.Conv2D(64, (3,3), padding='same', activation='relu')(conv2_2)
conv3_2 = layers.MaxPool2D()(conv3_1)

conv4_1 = layers.Conv2D(64, (3,3), padding='same', activation='relu')(conv3_2)
conv4_2 = layers.MaxPool2D()(conv4_1)

conv5_1 = layers.Conv2D(128, (3,3), padding='same', activation='relu')(conv4_2)
conv5_2 = layers.MaxPool2D()(conv5_1)

conv6_1 = layers.Conv2D(128, (3,3), padding='same', activation='relu')(conv5_2)
conv6_2 = layers.MaxPool2D()(conv6_1)

flatten = layers.Flatten()(conv6_2)
dense1 = layers.Dense(128, activation='relu')(flatten)
dense2 = layers.Dense(32, activation='relu')(dense1)
output_class = layers.Dense(1, activation='linear')(dense2)

resnet_model = models.Model(input_tensor, output_class)
resnet_model.summary()

# Loss Function

In [ ]:
import keras.backend as K

def L1_loss(new_true_val, new_pred_val):
    pass


def min_loss(new_true_val, new_pred_val):
    new_true_val = float(new_true_val)
    new_pred_val = float(new_pred_val)
    max_angle_range = tf.constant([360],dtype=float)
    if new_pred_val < 0 and new_pred_val > 360:
        return tf.abs(new_true_val - new_pred_val)**2
    else:
        # K.print_tensor('')
        # K.print_tensor(new_true_val, message='new_true_val = ')
        # K.print_tensor(new_pred_val, message='new_pred_val = ')
        error1 = tf.abs(new_true_val - new_pred_val)
        error2 = tf.abs(max_angle_range - tf.abs(new_true_val - new_pred_val))
        if tf.less(error1, error2)[0]:
            error = error1
        else:
            error = error2
        # K.print_tensor(error, message='error = ')
        return error**2


def sin_loss(new_true_val, new_pred_val): # sin Loss⇒sin((x2-x1)/2), 
    pi_on_180 = 0.017453292519943295 # convert val to radian
    new_thres_val = 0.1
    error = tf.abs(new_true_val - new_pred_val) * pi_on_180
    error = tf.sin(error/2)
    small_err = new_thres_val <= tf.abs(error)
    new_err_loss = tf.abs(error)*360
    max_err_loss = new_thres_val * tf.abs(error)
    return new_err_loss**2
    #return tf.where(small_err, new_err_loss, max_err_loss)


def cos_loss(new_true_val, new_pred_val): # cos Loss⇒|-cos((x2-x1)/2)+1|
    pi_on_180 = 0.017453292519943295 # convert val to radian
    new_thres_val = 0.1
    error1 = (new_true_val - new_pred_val) * pi_on_180
    error =  -tf.cos((error1/2)) + 1
    small_err = new_thres_val <= tf.abs(error)
    new_err_loss = tf.abs(error)
    max_err_loss = new_thres_val * tf.abs(error)
    return tf.where(small_err, new_err_loss, max_err_loss)

In [ ]:
pi_on_180 = 0.017453292519943295 # convert val to radian
temp = tf.convert_to_tensor([0, 90.,180., 270., 360.])
print(temp*pi_on_180)
print(tf.cos(temp * pi_on_180)+1)


pi_on_180 = 0.017453292519943295 # convert val to radian
temp = tf.convert_to_tensor([0, 90.,180., 270., 360.])
print(temp*pi_on_180)
print(tf.sin(temp * pi_on_180))

# Train

In [ ]:
resnet_model.compile(optimizer='Adam', loss=sin_loss) # loss='mse'

In [ ]:
history = resnet_model.fit(
    train_data,
    epochs=50,
    verbose=2,
)

# history = res_model.fit(
#     train_images, 
#     validation_data=val_images, 
#     epochs=100,
#     callbacks=[
#           tf.keras.callbacks.EarlyStopping(
#               monitor='val_loss',
#               patience=5,
#               restore_best_weights=True
#         )
#     ]
# )

Epoch 1/50
360/360 - 23s - loss: 1449.8998 - 23s/epoch - 64ms/step
Epoch 2/50
360/360 - 22s - loss: 498.3393 - 22s/epoch - 62ms/step
Epoch 3/50
360/360 - 22s - loss: 2428.2400 - 22s/epoch - 61ms/step
Epoch 4/50
360/360 - 22s - loss: 652.1965 - 22s/epoch - 61ms/step
Epoch 5/50
360/360 - 22s - loss: 710.5201 - 22s/epoch - 61ms/step
Epoch 6/50
360/360 - 23s - loss: 1103.1115 - 23s/epoch - 65ms/step
Epoch 7/50
360/360 - 23s - loss: 1222.5566 - 23s/epoch - 63ms/step
Epoch 8/50
360/360 - 22s - loss: 1958.3344 - 22s/epoch - 61ms/step
Epoch 9/50
360/360 - 22s - loss: 509.2253 - 22s/epoch - 62ms/step
Epoch 10/50
360/360 - 22s - loss: 264.2053 - 22s/epoch - 61ms/step
Epoch 11/50
360/360 - 22s - loss: 1447.0382 - 22s/epoch - 61ms/step
Epoch 12/50
360/360 - 22s - loss: 2702.2493 - 22s/epoch - 61ms/step
Epoch 13/50
360/360 - 22s - loss: 2931.2815 - 22s/epoch - 61ms/step
Epoch 14/50
360/360 - 22s - loss: 3468.7478 - 22s/epoch - 61ms/step
Epoch 15/50
360/360 - 23s - loss: 1973.9950 - 23s/epoch - 63ms

In [ ]:
resnet_model.save(model_path)

# Result

In [ ]:
prediction_value = np.squeeze(resnet_model.predict(test_data))
true_value = test_data.labels

rmse = np.sqrt(resnet_model.evaluate(test_data, verbose=0))
print("Test RMSE : {:.5f}".format(rmse))

r2 = r2_score(true_value, prediction_value)
print("Test R^2 Score: {:.5f}".format(r2))

Test RMSE : 34.93952
Test R^2 Score: -4.63241


In [ ]:
# null_rmse = np.sqrt(np.sum((true_value - np.mean(true_value))**2) / len(true_value))
# print("Null/Baseline Model Test RMSE: {:.5f}".format(null_rmse))

In [ ]:
import cv2

for i in range(100):
  temp = train_images.Filepath[i]
  split_temp = temp.split("/")[2]
  print('idx :',i, ', y :',split_temp, end="  ")
  img2 = cv2.imread(temp)
  # int -> float
  scalingFactor = 1
  # Convert unsigned int 8bit to float
  img2 = np.float32(img2)
  img2 = img2*scalingFactor
  # input layer 형식 맞추기
  img2 = cv2.resize(img2, (height, width))

  img2 = img2.reshape(-1, height, width,3)
  # print(4,img2.shape)
  y_hat = np.squeeze(resnet_model.predict(img2))
  print(' y_hat :',y_hat%360)

idx : 0 , y : 145   y_hat : 144.8155059814453
idx : 1 , y : 1   y_hat : 5.992748260498047
idx : 2 , y : 336   y_hat : 338.23114013671875
idx : 3 , y : 138   y_hat : 139.75759887695312
idx : 4 , y : 328   y_hat : 330.69158935546875
idx : 5 , y : 112   y_hat : 115.23977661132812
idx : 6 , y : 353   y_hat : 358.62070965766907
idx : 7 , y : 249   y_hat : 249.0749969482422
idx : 8 , y : 216   y_hat : 218.49591064453125
idx : 9 , y : 148   y_hat : 148.1342010498047
idx : 10 , y : 190   y_hat : 213.9866943359375
idx : 11 , y : 230   y_hat : 232.92768096923828
idx : 12 , y : 40   y_hat : 41.39617919921875
idx : 13 , y : 56   y_hat : 59.146488189697266
idx : 14 , y : 116   y_hat : 121.06489562988281
idx : 15 , y : 197   y_hat : 197.55813598632812
idx : 16 , y : 264   y_hat : 265.6064910888672
idx : 17 , y : 358   y_hat : 359.4808980822563
idx : 18 , y : 333   y_hat : 335.2097797393799
idx : 19 , y : 73   y_hat : 74.29496002197266
idx : 20 , y : 137   y_hat : 136.8545379638672
idx : 21 , y : 75 

In [ ]:
img2.shape

(1, 200, 200, 1)

# Model Save

In [ ]:
resnet_model.save('my_model_sin_loss.h5')